In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import evaluate
from evaluate import load
from datasets import load_dataset

In [3]:
data = []
with open("Paper-Reproduction-Speculative-Decoding\\Results\\results2.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [4]:
humanEval = load_dataset("openai/openai_humaneval",trust_remote_code=True)["test"]
ds=humanEval.select(range(100))

In [10]:
acceptance_ratio=[]
speedup=[]

for i in range(100):
    acceptance_ratio.append(data[i]['acceptance_ratio'])
    speedup.append(data[i]['speculative_speed']/data[i]['inference_speed'])

print(f'Speed up= {np.array(speedup).mean()}        Acceptance rate= {np.array(acceptance_ratio).mean()}')

Speed up= 2.5249470557715616        Acceptance rate= 74.32718933553865


In [11]:
def check_correctness(code, tests):
    namespace = {}
    namespace['input'] = lambda *args, **kwargs: ""
    try:
        # Add basic required imports
        exec("import math\nimport random\nimport string", namespace)
        exec(code, namespace)
        exec(tests, namespace)
        return True
    except Exception as e:
        print(f"Test failed with error: {e}")  # Enable error printing
        return False

# Initialize counters and tracking
original_passed = 0
speculative_passed = 0
total_processed = 0
skipped_entries = 0

# Calculate pass@1 scores
for i in range(min(len(data), len(ds))):
    try:
        print(f"\nProcessing entry {i}:")
        original_code = data[i].get('original_code', None)
        speculative_code = data[i].get('speculative_code', None)
        test_case = ds[i].get('test', None)
        
        if not all([original_code, speculative_code, test_case]):
            print(f"Skipping entry {i} - Missing required fields")
            skipped_entries += 1
            continue
            
        total_processed += 1
        
        # Run tests and track results
        if check_correctness(original_code, test_case):
            original_passed += 1
            print(f"Entry {i}: Original passed")
        else:
            print(f"Entry {i}: Original failed")
            
        if check_correctness(speculative_code, test_case):
            speculative_passed += 1
            print(f"Entry {i}: Speculative passed")
        else:
            print(f"Entry {i}: Speculative failed")
            
    except Exception as e:
        print(f"Error processing entry {i}: {e}")
        skipped_entries += 1

# Print detailed results
print("\n=== Test Run Summary ===")
print(f"Total entries available: {len(data)}")
print(f"Total entries processed: {total_processed}")
print(f"Skipped entries: {skipped_entries}")
print(f"Original model pass@1: {(original_passed/total_processed)*100:.2f}% ({original_passed}/{total_processed})")
print(f"Speculative model pass@1: {(speculative_passed/total_processed)*100:.2f}% ({speculative_passed}/{total_processed})")


Processing entry 0:
Entry 0: Original passed
Test failed with error: invalid syntax (<string>, line 21)
Entry 0: Speculative failed

Processing entry 1:
Entry 1: Original passed
Entry 1: Speculative passed

Processing entry 2:
Test failed with error: unterminated triple-quoted string literal (detected at line 40) (<string>, line 40)
Entry 2: Original failed
Test failed with error: invalid syntax (<string>, line 39)
Entry 2: Speculative failed

Processing entry 3:
Entry 3: Original passed
Test failed with error: invalid syntax (<string>, line 23)
Entry 3: Speculative failed

Processing entry 4:
Test failed with error: expected ':' (<string>, line 31)
Entry 4: Original failed
Test failed with error: expected ':' (<string>, line 27)
Entry 4: Speculative failed

Processing entry 5:
Test failed with error: unterminated triple-quoted string literal (detected at line 32) (<string>, line 32)
Entry 5: Original failed
Entry 5: Speculative passed

Processing entry 6:
Entry 6: Original passed
Ent